In [ ]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

In [ ]:
import numpy as np
import pandas as pd
import datetime as dt

# Reflect Tables into SQLAlchemy ORM

In [ ]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

In [ ]:
# create engine to hawaii.sqlite
engine = create_engine("sqlite:///Resources/hawaii.sqlite")

In [ ]:
# reflect an existing database into a new model
base = automap_base()

# reflect the tables
base.prepare(engine, reflect = True)

In [ ]:
# View all of the classes that automap found
base.classes.keys()

In [ ]:
# Save references to each table
measurement = base.classes.measurement
Station = base.classes.station

In [ ]:
# Create our session (link) from Python to the DB
session = Session(engine)

# Exploratory Precipitation Analysis

In [ ]:
# Find the most recent date in the data set.
recent_date = session.query(measurement.date).order_by(measurement.date.desc()).first()
print(recent_date)

In [ ]:
# Design a query to retrieve the last 12 months of precipitation data and plot the results. 
# Starting from the most recent data point in the database. 

# Calculate the date one year from the last date in data set.
last_year_date = dt.date(2017, 8, 23) - dt.timedelta(days = 365)
print(last_year_date)

# Perform a query to retrieve the data and precipitation scores
rain_scores = session.query(measurement.date, measurement.prcp).filter(measurement.date >= last_year_date).all()

# Save the query results as a Pandas DataFrame. Explicitly set the column names
rain_data = pd.dataframe()
rain_data = rain_data.rename(columns = {"date", "precipitation"})

# Sort the dataframe by date
rain_data.sort_values(by = ["date"], ascending = True)
rain_data = rain_data.dropna()

# Use Pandas Plotting with Matplotlib to plot the data
rain_data.plot(x = "precipitation", grid = True, legend = True)
plt.xlabel("Date")
plt.ylabel("Inches")
plt.show()


In [ ]:
# Use Pandas to calculate the summary statistics for the precipitation data
rain_data.describe()

# Exploratory Station Analysis

In [ ]:
# Design a query to calculate the total number of stations in the dataset
stations = session.query(func.count(measurement.station)).all()

In [ ]:
# Design a query to find the most active stations (i.e. which stations have the most rows?)
# List the stations and their counts in descending order.
most_active = session.query(measurement.station, func.count(measurement.station)).group_by(measurement.station)
                    group_by(measurement.station)
                    order_by(func.count(measurement.station).desc()).all()
most_active

In [ ]:
# Using the most active station id from the previous query, calculate the lowest, highest, and average temperature.
station_temps = session.query(func.min(measurement.tobs), func.max(measurement.tobs), func.avg(measurement.tobs))
                    filter(measurement.station == most_active[0][0]).all()

In [ ]:
# Using the most active station id
# Query the last 12 months of temperature observation data for this station and plot the results as a histogram
most_active_results = session.query(measurement.tobs).filter(measurement.station == most_active)
                    filter(measurement.date >= last_year_date).all()

most_active_df = pd.dataframe(most_active_results, columns = ["tobs"])
most_active_df.hist(bins = 12, grids = True, legend = True)
plt.ylabel = "Frequency"
plt.xlabel = "Temperature"

# Close Session

In [ ]:
# Close Session
session.close()